# Imports and Definitions

In [ ]:
import os

In [ ]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

In [ ]:
paths = {
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'IMAGES_PATH': os.path.join('Tensorflow', 'workspace','images'),
}

for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
files = {
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
}

# Installations

In [ ]:
# Wget
!pip install wget

In [ ]:
# Tensorflowjs (for exporting)
!pip install tensorflowjs

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

# Download Models

Run this section whenever you want to add a new model for training. 

Bug: Issue with path['PRETRAINED_MODEL_PATH'], hard coded path

In [ ]:
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    #!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} Tensorflow/workspace/pre-trained-models
    !cd Tensorflow/workspace/pre-trained-models && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    #!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} Tensorflow/workspace/pre-trained-models
    !cd Tensorflow/workspace/pre-trained-models && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# Download Test Data

Downloads a set of 50 images (40 training, 10 test) for model training. The 'Tensorflow/workspace/images' must be empty.

In [ ]:
!git clone https://github.com/ary1111/login_training_dataset {paths['IMAGES_PATH']}